In [30]:
from pathlib import Path
import argparse
import time
import random
import traceback

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

# Destination fixed by assignment
FIXED_RECIPIENT = "scittest@auditram.com"

# screenshot folder
OUT_DIR = Path.cwd() / "email_automator_outputs"
OUT_DIR.mkdir(exist_ok=True)


# ---------------------------
# Utilities
# ---------------------------
def jitter_sleep(base=0.35, jitter=0.15):
    """Sleep a little with random jitter to look less robotic."""
    time.sleep(base + random.uniform(-jitter, jitter))


def snapshot(driver, name):
    path = OUT_DIR / name
    try:
        driver.save_screenshot(str(path))
        print(f"[SAVE] Screenshot: {path}")
    except Exception as e:
        print(f"[WARN] Could not save screenshot: {e}")


# ---------------------------
# Robust element helpers
# ---------------------------
def click_if_available(wait, candidates):
    """Try clicking the first clickable element among candidate (By, selector) tuples."""
    for by, sel in candidates:
        try:
            el = wait.until(EC.element_to_be_clickable((by, sel)))
            el.click()
            return True
        except Exception:
            continue
    return False


def locate_visible(driver, selectors, timeout=12):
    """Return first visible element found for any (By, selector) in selectors list."""
    end = time.time() + timeout
    while time.time() < end:
        for by, sel in selectors:
            try:
                els = driver.find_elements(by, sel)
                for e in els:
                    if e.is_displayed() and e.is_enabled():
                        return e
            except Exception:
                continue
        time.sleep(0.3)
    return None


def set_value_js(driver, element, text):
    """Set value for inputs/contenteditable via JS and dispatch events."""
    script = """
      const el = arguments[0];
      const val = arguments[1];
      try {
        if (el.isContentEditable) {
          el.focus();
          el.innerText = val;
        } else {
          el.focus();
          el.value = val;
        }
        el.dispatchEvent(new Event('input', {bubbles:true}));
        el.dispatchEvent(new Event('change', {bubbles:true}));
        return true;
      } catch (err) {
        return false;
      }
    """
    try:
        return driver.execute_script(script, element, text)
    except Exception:
        return False


# ---------------------------
# Browser / login flow
# ---------------------------
def build_driver(use_fresh_profile: bool):
    opts = Options()
    opts.add_argument("--start-maximized")
    # avoid headless: visible browser is required
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    opts.add_argument("--disable-blink-features=AutomationControlled")

    if use_fresh_profile:
        # store a temporary user-data-dir inside working folder
        profile_path = Path.cwd() / ".temp_chrome_profile_email_automator"
        opts.add_argument(f"--user-data-dir={str(profile_path)}")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    # reasonable implicit wait
    driver.implicitly_wait(6)
    return driver


def perform_login(driver, wait, user_email, user_password):
    """
    Attempt to log in to Google Mail using multiple fallback methods.
    Returns True on detection of inbox, False otherwise.
    """

    # open Google accounts login for mail
    driver.get("https://accounts.google.com/ServiceLogin?service=mail&continue=https://mail.google.com/")
    jitter_sleep(0.6)

    # 1) Enter email/identifier
    try:
        id_elem = wait.until(EC.presence_of_element_located((By.ID, "identifierId")))
        id_elem.clear()
        id_elem.send_keys(user_email)
        jitter_sleep()
        click_if_available(wait, [
            (By.ID, "identifierNext"),
            (By.XPATH, "//button//span[text()='Next']/..")
        ]) or id_elem.send_keys(Keys.ENTER)
        print("[INFO] Email submitted.")
    except Exception as e:
        print("[ERROR] Could not submit email:", e)
        snapshot(driver, "login_no_email_field.png")
        return False

    jitter_sleep(0.8)

    # try to find password field using several strategies
    pwd_selectors = [
        (By.XPATH, "//input[@type='password']"),
        (By.NAME, "password"),
        (By.XPATH, "//input[contains(@aria-label,'password') or contains(@placeholder,'Password')]"),
    ]
    password_el = locate_visible(driver, pwd_selectors, timeout=20)

    # If not found, try clicking "Use another account" or similar flows then search again
    if not password_el:
        try:
            alternatives = driver.find_elements(By.XPATH, "//*[contains(text(),'Use another account') or contains(text(),'Add account')]")
            for a in alternatives:
                try:
                    if a.is_displayed():
                        a.click()
                        jitter_sleep(0.6)
                        break
                except Exception:
                    continue
        except Exception:
            pass
        password_el = locate_visible(driver, pwd_selectors, timeout=12)

    if not password_el:
        print("[WARN] Password field not found automatically; possible extra verification (2FA/captcha).")
        snapshot(driver, "login_no_password_field.png")
        return False

    # Try typing the password, fallback to JS if send_keys fails
    try:
        password_el.click()
        jitter_sleep(0.12)
        password_el.clear()
        password_el.send_keys(user_password)
        print("[INFO] Password typed via send_keys.")
    except Exception as e:
        print("[WARN] send_keys failed for password field; trying JS method. Error:", e)
        ok = set_value_js(driver, password_el, user_password)
        if not ok:
            print("[ERROR] JS method also failed to set password.")
            snapshot(driver, "login_pwd_set_failed.png")
            return False
        else:
            print("[INFO] Password set via JS fallback.")

    # submit password: try several buttons or press Enter
    clicked = click_if_available(wait, [
        (By.ID, "passwordNext"),
        (By.XPATH, "//button//span[text()='Next']/.."),
        (By.XPATH, "//div[@id='passwordNext']"),
    ])
    if not clicked:
        try:
            password_el.send_keys(Keys.ENTER)
        except Exception:
            pass

    # wait for inbox presence
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[text()='Compose' or @aria-label='Compose']")), timeout=30)
        print("[INFO] Inbox detected.")
        return True
    except Exception:
        print("[WARN] Inbox not detected after login; screenshot saved.")
        snapshot(driver, "login_inbox_not_detected.png")
        return False


# ---------------------------
# Compose & send
# ---------------------------
def open_compose_and_send(driver, wait, subject_text, body_text):
    # open compose by URL for higher reliability
    composer_url = "https://mail.google.com/mail/u/0/?view=cm&fs=1&to=" + FIXED_RECIPIENT
    try:
        driver.get(composer_url)
        jitter_sleep(0.8)
    except Exception:
        pass

    # subject field — try several selectors
    subj_el = locate_visible(driver, [
        (By.NAME, "subjectbox"),
        (By.NAME, "subject"),
        (By.XPATH, "//input[@aria-label='Subject']"),
    ], timeout=12)

    if subj_el:
        try:
            subj_el.click()
            subj_el.clear()
            subj_el.send_keys(subject_text)
            print("[INFO] Subject set.")
        except Exception:
            print("[WARN] Could not set subject via send_keys; continuing.")
    else:
        print("[WARN] Subject element not located; continuing.")

    # body: try to find contenteditable
    body_el = locate_visible(driver, [
        (By.XPATH, "//div[@aria-label='Message Body' and @contenteditable='true']"),
        (By.XPATH, "//div[@role='textbox' and @contenteditable='true']"),
        (By.CSS_SELECTOR, "div.Am.Al.editable"),
    ], timeout=15)

    if not body_el:
        print("[ERROR] Could not find message body area. Composer may not have loaded.")
        snapshot(driver, "compose_body_not_found.png")
        return False

    # fill body (try send_keys then JS)
    try:
        body_el.click()
        jitter_sleep(0.12)
        body_el.send_keys(body_text)
        print("[INFO] Body typed via send_keys.")
    except Exception:
        ok = set_value_js(driver, body_el, body_text)
        if ok:
            print("[INFO] Body set via JS fallback.")
        else:
            print("[ERROR] Failed to set email body.")
            snapshot(driver, "compose_body_set_failed.png")
            return False

    jitter_sleep(0.4)

    # attempt to send using several selectors, then Ctrl+Enter
    sent = click_if_available(wait, [
        (By.XPATH, "//div[@role='button' and (contains(@data-tooltip,'Send') or contains(@aria-label,'Send'))]"),
        (By.XPATH, "//button[contains(@aria-label,'Send') or contains(@data-tooltip,'Send')]"),
        (By.XPATH, "//div[text()='Send']"),
    ])

    if not sent:
        try:
            body_el.send_keys(Keys.CONTROL, Keys.ENTER)
            sent = True
            print("[INFO] Sent via Ctrl+Enter fallback.")
        except Exception as e:
            print("[ERROR] Send failed:", e)

    if sent:
        snapshot(driver, "send_attempt.png")
        print("[SUCCESS] Send triggered (verify Sent folder if needed).")
        return True
    else:
        snapshot(driver, "send_failed.png")
        return False


# ---------------------------
# Main CLI flow
# ---------------------------
def parse_cli():
    p = argparse.ArgumentParser(description="Email automator (assignment)")
    p.add_argument("--email", required=True, help="Gmail address")
    p.add_argument("--password", required=True, help="Gmail password or app password")
    p.add_argument("--subject", required=True, help="Email subject")
    p.add_argument("--body", required=True, help="Email body")
    p.add_argument("--fresh-profile", action="store_true", help="Use dedicated temporary Chrome profile directory")
    return p.parse_args()


def main():
    args = parse_cli()

    driver = None
    try:
        driver = build_driver(use_fresh_profile=args.fresh_profile)
        wait = WebDriverWait(driver, 18)

        ok = perform_login(driver, wait, args.email, args.password)
        if not ok:
            print("[EXIT] Login did not complete automatically. Use an App Password or try with --fresh-profile.")
            return

        sent = open_compose_and_send(driver, wait, args.subject, args.body)
        if not sent:
            print("[EXIT] Compose/send step failed. Check saved screenshots in 'email_automator_outputs' for details.")
            return

    except Exception as exc:
        print("[ERROR] Unexpected exception:")
        traceback.print_exc()
        if driver:
            snapshot(driver, "unhandled_exception.png")
    finally:
        if driver:
            print("Cleaning up: closing browser in 4 seconds...")
            time.sleep(4)
            try:
                driver.quit()
            except Exception:
                pass


if __name__ == "__main__":
    main()


[INFO] Email submitted.
[INFO] Password typed via send_keys.
[WARN] Inbox not detected after login; screenshot saved.
[SAVE] Screenshot: C:\Users\bkapa\email_automator_outputs\login_inbox_not_detected.png
[EXIT] Login did not complete automatically. Use an App Password or try with --fresh-profile.
Cleaning up: closing browser in 4 seconds...
